# 13. Expanding your R Skills

Throughout this book we have covered some popular packages and specific functions from these packages. However, it would be impossible to cover all the packages, functions, and options that R has. As you start to apply the tools from this book to you own work or in new settings you may need new packages or encounter some unexpected errors. Practicing reading package documentation and responding to error messages will help you be able to expand your R skills beyond the topics covered here. 

We will demonstrate these skills using the `stringr` package, which is a package that is part of the tidyverse that has several functions for dealing with text data. 

In [1]:
suppressPackageStartupMessages(library(tidyverse))

## Reading Documentation for New Packages

Every published package has a CRAN website. This website contains a reference manual which contains the documentation for the functions and data available in the package. Most often there are also useful vignettes that give examples using the package. The site also tells you the requirements for using the package, the authors, and when it was last updated. Take a look at the CRAN site for [stringr](https://cran.r-project.org/web/packages/stringr/index.html) and read the vignette [Introduction to String R](https://cran.r-project.org/web/packages/stringr/vignettes/stringr.html). 

We will use this package to clean up text related to a PubMed search query for a systematic review. An example search query is given below and is taken from [Adatia et al. (2021). Out-of-hospital cardiac arrest: a systematic review of current risk scores to predict survival. American heart journal, 234, 31-41](https://www.sciencedirect.com/science/article/abs/pii/S0002870320304130). Our first goal will be to extract the actual search query from the text along with all the terms used in the query. We can assume that the search query will either be fully contained in parantheses or will be a sequence of paranthetical phrases connected with AND or OR. Our goal is to extract the search query as well as all the individual search terms used in the query, but we have to get there in a series of steps.

In [2]:
sample_str <- " A systematic search will be performed in PubMed, Embase, and the Cochrane Library, 
using the following search query:   ('out-of-hospital cardiac arrest' OR 'OHCA') AND ('MIRACLE 2' OR 'OHCA' OR 
'CAHP' OR 'C-GRAPH' OR 'SOFA' OR 'APACHE' OR 'SAPS’ OR ’SWAP’ OR ’TTM’)."

The first thing we want to do with the text is clean up the whitespace by remove any trailing, leading, or multiple spaces. In our example, the string starts with a trailing space and there are multiple spaces right before the search query. Searching for "whitespace" in the stringr reference manual we find the `str_trim()` and `str_squish()` functions. Read the documentation for these two functions. You should find that `str_squish()` is the function we are looking for and it takes a single arguments. 

In [3]:
sample_str <- str_squish(sample_str)
sample_str

[1] "A systematic search will be performed in PubMed, Embase, and the Cochrane Library, using the following search query: ('out-of-hospital cardiac arrest' OR 'OHCA') AND ('MIRACLE 2' OR 'OHCA' OR 'CAHP' OR 'C-GRAPH' OR 'SOFA' OR 'APACHE' OR 'SAPS’ OR ’SWAP’ OR ’TTM’)."

## Trying Simple Examples

Above is a good example of starting with a simple case. Rather than applying my function to my full data yet, I want to make sure I understand how it works on a simple example on which I can anticipate what the outcome should look like. My next task will be to split the text into words stored as a character vector. Read the documentation to determine why I used the `str_split_1()` function below. I double check that the returned result is indeed a vector and print the result.

In [4]:
sample_str_words <- str_split_1(sample_str, " ")
class(sample_str_words)
sample_str_words

[1] "character"

[1] "A"                 "systematic"        "search"           
 [4] "will"              "be"                "performed"        
 [7] "in"                "PubMed,"           "Embase,"          
[10] "and"               "the"               "Cochrane"         
[13] "Library,"          "using"             "the"              
[16] "following"         "search"            "query:"           
[19] "('out-of-hospital" "cardiac"           "arrest'"          
[22] "OR"                "'OHCA')"           "AND"              
[25] "('MIRACLE"         "2'"                "OR"               
[28] "'OHCA'"            "OR"                "'CAHP'"           
[31] "OR"                "'C-GRAPH'"         "OR"               
[34] "'SOFA'"            "OR"                "'APACHE'"         
[37] "OR"                "'SAPS’"            "OR"               
[40] "’SWAP’"            "OR"                "’TTM’)."

We now want to identify words in this vector that have a starting and/or end parantheses. The function `grepl()` takes in a character vector `x` and a pattern to search for. It returns a logical vector for whether or not each element of x has a match for that pattern.

In [5]:
grepl(sample_str_words, ")")

Warning message in grepl(sample_str_words, ")"):
“argument 'pattern' has length > 1 and only the first element will be used”


[1] FALSE

Huh, that didn't match what I expected! I expected to have multiple TRUE/FALSE values outputted - one for each word. Let's read the documentation again. 

## Deciphering Error Messages and Warnings

The warning message will give us a good clue for what went wrong. It says that the inputted pattern has length > 1. However, the pattern I gave it is a single character. In fact, I specified the arguments in the wrong order. Let's try again. This time I specify `x` and `pattern`. 

In [6]:
grepl(x=sample_str_words, pattern=")")

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
[25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[37] FALSE FALSE FALSE FALSE FALSE  TRUE

That worked! However, it won't work if we change that to a starting parantheses. Try it out for yourself. The error message says that it is looking for an end parantheses. In this case, the documentation does not help us. Let's try googling "stringr find start parantheses". The first search result for me is a [stack overflow question](https://stackoverflow.com/questions/56174805/how-to-search-for-strings-with-parentheses-in-r) that helps us out. We read that we need to use backslashes to tell R to read the parentheses literally rather than as a special character used in a regular expression. 

In [7]:
grepl(x=sample_str_words, pattern="\\(")

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE
[25]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[37] FALSE FALSE FALSE FALSE FALSE FALSE

When a function doesn't return what we expected it is a good idea to first test the arguments we gave it match what we expect and then to re-read the documentation. For example, we could check that `sample_str_words` is indeed a character vector. 

### Debugging Code 

The code below is supposed to extract the search query from the text as well as find the invididual search terms used in the query. However, the code is incorrect. Practice reading through the code to understand what it is trying to do. The comments are there to help explain the steps but you may also want to print the output to figure out what it is doing.

In [8]:
sample_str <- " A systematic search will be performed in PubMed, Embase, and the Cochrane Library, 
using the following search query:   ('out-of-hospital cardiac arrest' OR 'OHCA') AND ('MIRACLE 2' OR 'OHCA' OR 
'CAHP' OR 'C-GRAPH' OR 'SOFA' OR 'APACHE' OR 'SAPS’ OR ’SWAP’ OR ’TTM’)."

# separate parentheses, remove extra whitespace, and split into words
sample_str <- str_replace(sample_str, "\\)", " \\) ")
sample_str <- str_replace(sample_str, "\\(", " \\( ")
sample_str <- str_squish(sample_str)
sample_str_words <- str_split_1(sample_str, " ")

# find indices with parentheses 
end_ps <- grepl(x=sample_str_words, pattern="\\)")
start_ps <- grepl(x=sample_str_words, pattern="\\(")

# find words between first and last parentheses 
search_query <- sample_str_words[ which(end_ps)[1]:which(start_ps)[1]]
search_query <- paste(search_query, collapse=" ")
search_query

# find search terms
search_terms <- str_replace_all(search_query, "\\)", "")
search_terms <- str_replace_all(search_query, "\\(", "")
sample_terms <- str_squish(search_query)
search_terms <- str_split_1(search_terms, " AND | OR ")
search_terms

[1] ") 'OHCA' OR arrest' cardiac 'out-of-hospital ("

[1] ") 'OHCA'"                          "arrest' cardiac 'out-of-hospital "

## Video Solution

We will show how we can test and fix the code above by using some simple debugging principles. 

<div class="video-container">
    <iframe width="700" height="500" src="https://www.youtube.com/embed/2yfjXyAbbK8" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>
</div>

## General Programming Tips

As you write more complex code and functions, it is inevitable that you will run into errors or unexpected behavior. Below are some simple principles that are applicable to debugging in any setting. When it comes to testing code, a good mantra is test early and test often. So avoid writing too much code before running and checking that the results match what you expect. 

  1.  Check that all paranthesis (), brackets [], and curly braces {} match.  
  2.  Check that object names are correct.   
  3.  Check if you use the same name for different objects or that you use different names for the same object. You can do this by using the `ls()` function to find all current objects. 
  4.  Check that the input arguments to a function match what is expected.  
  5.  Try simple examples first. You can use the documentation or vignette examples for ideas. 
  6.  Localize your error by checking the values of objects at different points.  
  7.  Modify your code one piece at a time before checking it to avoid introducing new errors.  
  8.  Google error messages you don't understand. R's messages can sometimes hint at what the error might stem from but are not always direct.

## Exercises

TODO: two exercises - one with brackets and logic off and misuse function str_sub, one with arguments wrong for calibration plot (used wrong name)